# QUICKSTART

本节介绍了机器学习中常见任务的API。请参考每一节中的链接来深入了解。

## 与数据打交道

PyTorch有两个处理数据的基元：`torch.utils.data.DataLoader` 和 `torch.utils.data.Dataset`。`Dataset` 存储样本及其相应的标签，而 `DataLoader` 则围绕 `Dataset` 包装了一个可迭代的数据。

In [2]:
import torch  # 导入 torch
from torch import nn  # 从 torch 导入 nn 模块，该模块用于构建网络模型
from torch.utils.data import DataLoader  # 导入数据集加载器
from torchvision import datasets  # 导入视觉相关的数据集
from torchvision.transforms import ToTensor  # 从 torchvision 导入视觉相关变换的模块
from torch.utils.tensorboard import SummaryWriter  # 导入 tensorboard 用于可视化训练过程

In [2]:
writer = SummaryWriter('logs/log_2')  # 实例 SummaryWriter

PyTorch提供了特定领域的库，如 TorchText、TorchVision 和 TorchAudio，它们都包括数据集。在本教程中，我们将使用一个 TorchVision 数据集。

`torchvision.datasets` 模块包含了许多真实世界的视觉数据的 `Dataset` 对象，如 CIFAR、COCO（[完整列表在此](https://pytorch.org/vision/stable/datasets.html)）。在本教程中，我们使用 FashionMNIST 数据集。每个 TorchVision `Dataset` 都包括两个参数：`transform` 和 `target_transform`，分别用来修改样本和标签。

In [3]:
# 从开源数据集中下载训练集数据，命名为 training_data
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 从开源数据集中下载测试集数据，命名为 test_data
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


我们将 `Dataset` 对象作为参数传递给 `DataLoader`。这在我们的数据集上包装了一个可迭代的数据集，并支持自动批处理、采样、混淆和多进程数据加载。在这里，我们定义了一个 64 的批处理量，即 dataloader 可迭代的每个元素将返回 64 个特征和标签的批次。

In [4]:
# 设置数据集加载器每一批的数量
batch_size = 64

# 创建数据加载器，train_dataloader 和 test_dataloader
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# 以迭代器的方式遍历测试集加载器，并打印第一批数据和标签的形状
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.int64


了解更多关于[在 PyTorch 中加载数据](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html)。

## 创建模型

为了在 PyTorch 中定义一个神经网络，我们创建一个继承自 `nn.Module` 的类。我们在 `__init__` 函数中定义网络的层，并在 `forward` 函数中指定数据将如何通过网络。为了加速神经网络的操作，如果有条件的话，我们将其移到GPU上。

In [3]:
# 获取用于训练的 cpu 或 gpu 设备
# device = "cuda" if torch.cuda.is_available() else "cpu"
# print(f"Using {device} device")

# 定义模型
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# 实例化模型
model = NeuralNetwork()
model = nn.DataParallel(model.cuda())
print(model)

DataParallel(
  (module): NeuralNetwork(
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (linear_relu_stack): Sequential(
      (0): Linear(in_features=784, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=512, bias=True)
      (3): ReLU()
      (4): Linear(in_features=512, out_features=10, bias=True)
    )
  )
)


了解更多关于[在PyTorch中构建神经网络](https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html)。

## 优化模型参数

为了训练一个模型，我们需要一个损失函数和一个优化器。

In [20]:
# 定义交叉熵损失
loss_fn = nn.CrossEntropyLoss()

# 优化器选用随机梯度下降，学习率设置为 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


在一个单一的训练循环中，模型对训练数据集（分批送入）进行预测，并通过反向传播预测误差来调整模型的参数。

In [22]:
# 定义训练部分函数，传入 dataloader、model、loss_fn、optimizer、epoch 等参数
def train(dataloader, model, loss_fn, optimizer, epoch):
    # 获取 dataloader 的数据集尺寸
    size = len(dataloader.dataset)

    # 将模型设置为训练模式
    model.train()

    # 对 dataloader 分批进行迭代训练
    for batch, (X, y) in enumerate(dataloader):
        # 将数据部署在 device 上
        X, y = X.cuda(), y.cuda()

        # 计算预测损失
        pred = model(X)
        loss = loss_fn(pred, y)

        # 反向传播
        optimizer.zero_grad()  # 梯度清零
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数

        # 每 100 个批次输出一次训练日志
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}    [{current:>5d}/{size:>5d}]")
    writer.add_scalar('training_loss', loss, epoch)

我们还需根据测试数据集检查模型的性能，以确保它在进行有效地训练。

In [23]:
# 定义测试部分函数，传入 dataloader、model、loss_fn、epoch 等参数
def test(dataloader, model, loss_fn, epoch):
    # 获取 dataloader 的数据集尺寸
    size = len(dataloader.dataset)
    # 获取 dataloader 批次个数
    num_batches = len(dataloader)
    # 将模型设置为预测模式
    model.eval()
    # 初始化 test_loss、correct 为 0
    test_loss, correct = 0, 0
    # 在 torch 不自动计算梯度的条件下执行
    with torch.no_grad():
        # 对 dataloader 分批进行迭代预测
        for X, y in dataloader:
            # 将数据部署在 device 上
            X, y = X.cuda(), y.cuda()
            # 预测结果 pred
            pred = model(X)
            # 计算测试集当前批次的损失
            test_loss += loss_fn(pred, y)
            # 计算测试集当前批次的准确度
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches  # 计算测试集平均损失
    correct /= size  # 计算测试集平均准确度，由于准确度累加的是所有样本的结果，所以需要除以数据集总尺寸

    # 输出测试日志
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    writer.add_scalar('test_loss', test_loss, epoch)
    writer.add_scalar('accuracy', correct, epoch)

训练过程是通过几个迭代（epochs）进行的。在每个 epoch 中，模型学习参数以做出更好的预测。我们在每个 epoch 中打印模型的准确度和损失；我们希望看到准确度在每个 epoch 中增加，损失在每个 epoch 中减少。

In [24]:
# 定义 epochs
epochs = 15
# 迭代训练 epochs 次
for t in range(epochs):
    # 打印当前是第几个 epoch
    print(f"Epoch {t + 1}\n--------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, t)
    test(test_dataloader, model, loss_fn, t)

# 训练结束，关闭 writer
print("Done!")
writer.flush()

Epoch 1
--------------------------------
loss: 2.312794    [   64/60000]
loss: 2.290226    [ 6464/60000]
loss: 2.285262    [12864/60000]
loss: 2.271553    [19264/60000]
loss: 2.244183    [25664/60000]
loss: 2.234456    [32064/60000]
loss: 2.224790    [38464/60000]
loss: 2.229638    [44864/60000]
loss: 2.216705    [51264/60000]
loss: 2.189452    [57664/60000]
Test Error: 
 Accuracy: 43.7%, Avg loss: 2.169439 

Epoch 2
--------------------------------
loss: 2.160762    [   64/60000]
loss: 2.129711    [ 6464/60000]
loss: 2.167295    [12864/60000]
loss: 2.143079    [19264/60000]
loss: 2.075144    [25664/60000]
loss: 2.049515    [32064/60000]
loss: 2.026561    [38464/60000]
loss: 2.031681    [44864/60000]
loss: 1.980364    [51264/60000]
loss: 1.930782    [57664/60000]
Test Error: 
 Accuracy: 55.1%, Avg loss: 1.904456 

Epoch 3
--------------------------------
loss: 1.856879    [   64/60000]
loss: 1.849523    [ 6464/60000]
loss: 1.789931    [12864/60000]
loss: 1.808090    [19264/60000]
loss:

## 保存模型

保存模型的一个常见方法是序列化内部状态字典（包含模型参数）。

In [11]:
# 保存模型参数
torch.save(model.state_dict(), "checkpoints/model2.pth")
print("Saved PyTorch Model State to model2.pth")

Saved PyTorch Model State to model.pth


这个模型现在可以用来进行预测。

In [26]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
model.eval()
with torch.no_grad():
    for i in range(50):
        x, y = test_data[i][0], test_data[i][1]
        pred = model(x.cuda())
        predicted, actual = classes[pred[0].argmax(0)], classes[y]
        print(f'Predicted: "{predicted}", Actual: "{actual}", Result: {predicted == actual}')

Predicted: "Ankle boot", Actual: "Ankle boot", Result: True
Predicted: "Pullover", Actual: "Pullover", Result: True
Predicted: "Trouser", Actual: "Trouser", Result: True
Predicted: "Trouser", Actual: "Trouser", Result: True
Predicted: "Shirt", Actual: "Shirt", Result: True
Predicted: "Trouser", Actual: "Trouser", Result: True
Predicted: "Coat", Actual: "Coat", Result: True
Predicted: "Coat", Actual: "Shirt", Result: False
Predicted: "Sneaker", Actual: "Sandal", Result: False
Predicted: "Sneaker", Actual: "Sneaker", Result: True
Predicted: "Pullover", Actual: "Coat", Result: False
Predicted: "Sandal", Actual: "Sandal", Result: True
Predicted: "Sandal", Actual: "Sneaker", Result: False
Predicted: "Dress", Actual: "Dress", Result: True
Predicted: "Coat", Actual: "Coat", Result: True
Predicted: "Trouser", Actual: "Trouser", Result: True
Predicted: "Pullover", Actual: "Pullover", Result: True
Predicted: "Pullover", Actual: "Coat", Result: False
Predicted: "Bag", Actual: "Bag", Result: True
